In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('../data/preprocessed_train.csv')

In [3]:
train_df.shape

(17690, 896)

In [4]:
count_cols = [col for col in train_df.columns if 'count' in col]

In [6]:
for col in count_cols:
    train_df[col] = train_df[col] / train_df['accumulated_actions']

In [3]:
train_df['accuracy_group'].value_counts(normalize=True)

3    0.500000
0    0.239062
1    0.136292
2    0.124647
Name: accuracy_group, dtype: float64